In [ ]:
import os
import re
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
import xml.etree.ElementTree as ET

xml_data = open('donnees_appr_dev/test.xml', 'r').read()  # Read file
root = ET.XML(xml_data)  # Parse XML

data_train = []
cols = []
for i, child in enumerate(root):
    data_train.append([subchild.text for subchild in child])

df = pd.DataFrame(data_train)
df.columns = ['movie','review_id','name','user_id','note','commentaire']

In [ ]:
df = pd.read_csv("complete_preprocessing.csv")

In [ ]:
g = pd.DataFrame()
g['note'] = df['note'].replace(',','.')
for i in range(len(df)):
    g['note'][i] = g['note'][i].replace(',', '.')

In [ ]:
df

In [ ]:
df['note']=g['note']

In [ ]:
df['note'] = df['note'].astype(float)

In [ ]:
#transformer des notes de leurs forme à la forme [0,1,2,3,4,5,6,7,8,9]
df['note'] = df['note'].apply(lambda note: int((note * 2)-1))

In [ ]:
url_regex = "((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*"

In [ ]:
df['commentaire'] = df['commentaire'].str.replace(url_regex, ' ')

In [ ]:
to_remove = []
for i in range(len(df)):
    if df['commentaire'].iloc[i] != df['commentaire'].iloc[i] or df['commentaire'].iloc[i] == '' or df['commentaire'].iloc[i] == None :
        to_remove.append(i)
df.drop(index=to_remove,inplace=True)
df.reset_index()

In [ ]:
df = df.reset_index()
df.drop(columns=['index'],inplace=True)

In [ ]:
import emoji

In [ ]:
import emoji
def text_has_emoji(text):
    for character in text:
        if character in emoji.EMOJI_DATA:
            return True
    return False

In [ ]:
'''to_remove = []
for i in range(len(df)):
    if text_has_emoji(df['commentaire'].iloc[i]):
        to_remove.append(i)
df.drop(index=to_remove,inplace=True)
df.reset_index()'''

In [ ]:
df = df.reset_index()
df.drop(columns=['index'],inplace=True)

In [ ]:
from num2words import num2words
import re

In [ ]:
list_string = ['140 dollars','3 pommes 4 oranges 1000023 poires ']
output_string = [re.sub('(\d+)', lambda m: num2words(m.group(),lang='fr'), sentence) for sentence in list_string]

In [ ]:
output_string

In [ ]:
df

In [ ]:
list_string = df['commentaire'].values.tolist()
output_string = [re.sub('(\d+)', lambda m: num2words(m.group(),lang='fr'), sentence) for sentence in list_string]

In [ ]:
output_string

In [ ]:
df['commentaire'] = output_string

In [ ]:
df

In [ ]:
import re
import string

def remove_punctuation(text):
  # Use a regular expression to remove all punctuation except the single quote character
  text = re.sub("[%s]" % re.escape(string.punctuation.replace("'", "")), '', text)
  return text

text = "This is a test sentence with punctuation. ' Can you help me modify the code?"
text = remove_punctuation(text)
print(text)

In [ ]:
from bs4 import BeautifulSoup
import os
import re
import string
import json
import emoji

In [ ]:
punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "´": "'", "°": "", "€": "euros", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "pounds", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':'' ,'à':'a','â':'a','é':'e','è':'e','ê':'e',
                 'ë':'e','ç':'c','î':'i','ï':'i','ô':'o','û':'u','ù':'u','ü':'u','ÿ':'y','æ':'ae','œ':'oe','À':'A',
                'Â':'A','Ç':'C','É':'E','È':'E','Ê':'E','Ë':'E','Î':'I','Ï':'I','Ô':'O','Û':'U','Ù':'U','Ü':'U','Ÿ':'Y',
                'Æ':'AE','Œ':'OE'}

In [ ]:
def clean_text(text):
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Transformer les lettres de Majuscules en Minuscules
    text = re.sub('\[.*?\]', '', text)
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r"[^a-zA-ZàâçéèêëîïôûùüÿæœÀÂÇÉÈÊËÎÏÔÛÙÜŸÆŒ?.!,¿']+", " ", text) #garder pour ne pas enlever les charactères français
    #text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text) # décommenter pour enlever les charactères français
    return text
def clean_contractions(text):
    specials = ["’","‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # commenter cette ligne pour garder les charactère français''
    #text = re.sub("[%s]" % re.escape(string.punctuation.replace("'", "")), '', text)
    # creation d'un espace entre un mot et une ponctuatuion qui le suit
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text
def clean_special_chars(text, punct, mapping):  
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text
def remove_space(text):  
    text = text.strip()
    text = text.split()
    return " ".join(text)
'''def spellingCorrector(misspelled):
    correction = []
    misspelled = nltk.word_tokenize(misspelled)
    for word in misspelled:
        if re.search(r"'", word):
            correction.append(word)
        else:
            corr = spell.correction(word)
            if corr != None:
                correction.append(corr)
            else:
                correction.append(word)
    return ' '.join(correction)'''
def text_preprocessing_pipeline(text):
    text = clean_text(text)
    text = clean_contractions(text)
    #text = clean_special_chars(text, punct, punct_mapping) # décommenter pour enlever tous les ponctuations et 
    #charactères français
    text = remove_space(text)
    #text = spellingCorrector(text) # Computationellement forte de décommenter pas
    return text

In [ ]:
df_hj = df.commentaire.apply(lambda commentaire: ''.join(text_preprocessing_pipeline(commentaire)))

In [ ]:
df_hj

In [ ]:
df['commentaire'] = df_hj

In [ ]:
df.to_csv("dev_complete_preprocessing_2.csv")

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('french')

In [ ]:
for i,word in enumerate(stop):
    stop[i] = text_preprocessing_pipeline(word)

In [ ]:
stop

In [ ]:
dfstp = df['commentaire'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
dfstp

In [ ]:
df['commentaire'] = dfstp

In [ ]:
print([word for word in df['commentaire'].iloc[0].split() if word not in (stop)])

In [ ]:
df['word_count'] = df['commentaire'].apply(lambda x: len(str(x).split()))
plt.figure(figsize=(5,5))
sns.histplot(data=df, x='word_count')
plt.title('Nombre de mots dans les commentaires')
plt.xlabel('Nombre de mots')
plt.ylabel('Nombre de commentaires')
plt.show()

In [ ]:
plt.savefig('Nombre de mots dans les commentaires.png')

In [ ]:
test_articles = (sum(df['word_count'] < 360)/df.shape[0])*100
print("Pourcentage d'articles ayant moins de 360 mots:{:.2f}%".format(test_articles))